In [ ]:
# Install dependencies
%pip install --upgrade pip 
%pip install pandas requests

In [ ]:
import requests
import xml.etree.ElementTree as ET
import os

In [ ]:
input_url = "https://sitservicios.lapaz.bo/geoserver/ows"
data_dir = "../data"

In [ ]:
r = requests.get(f"{input_url}?service=WFS&version=1.0.0&request=GetCapabilities")
tree = ET.fromstring(r.content)

In [ ]:
ns = {"wfs": "http://www.opengis.net/wfs"}
layer_names = [el.text for el in tree.findall(".//wfs:Name", ns)]

In [ ]:
def create_metadata(layer_name):
    r = requests.get(
        f"{input_url}?service=WFS&version=1.0.0&request=DescribeFeatureType&typeName={layer_name}"
    )
    tree = ET.fromstring(r.content)

    metadata = {}
    for element in tree.findall(
        ".//xsd:element", {"xsd": "http://www.w3.org/2001/XMLSchema"}
    ):
        name = element.get("name")
        type_ = element.get("type")
        metadata[name] = type_

    return metadata

In [ ]:
import hashlib
import time
import json
from datetime import datetime, timezone

for name in layer_names:
    print(f"Descargando: {name}")
    wfs_url = f"{input_url}?service=WFS&version=1.0.0&request=GetFeature&typeName={name}&outputFormat=application/json"
    try:
        resp = requests.get(wfs_url)
        if resp.status_code == 200:
            filename = name.replace("lapaz:", "") + ".geojson"
            full_path = f"{data_dir}/{filename}"
            data = json.loads(resp.text)
            with open(full_path, "w", encoding="utf-8") as file_out:
                json.dump(data, file_out, indent=4, ensure_ascii=False)

            # Crear archivo de metadata
            with open(full_path, "rb") as f_in:
                file_contents = f_in.read()
            file_hash = hashlib.sha256(file_contents).hexdigest()
            metadata = {
                "hash": file_hash,
                "source": wfs_url,
                "request_status": resp.status_code,
                "unix_timestamp": int(time.time()),
                "timestamp": datetime.now(timezone.utc).isoformat(),
                "xml_metadata": create_metadata(name),
            }
            metadata_filename = name.replace("lapaz:", "") + ".metadata.json"
            with open(
                f"{data_dir}/{metadata_filename}", "w", encoding="utf-8"
            ) as meta_file:
                json.dump(metadata, meta_file, indent=4, ensure_ascii=False)
        else:
            print(f"❌ Error al descargar {name}")
    except Exception as e:
        print(f"⚠️ Excepción con {name}: {e}")